In [1]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

# 📂 Cartella con i file .pt
pt_dir = "/content/drive/MyDrive/NeuroOnco/Derivate"
serie_attese = ["T1W", "FLAIR", "DWI", "KTRAN", "CBV", "APT"]

# 🧾 Lista dei record da visualizzare
records = []

# 🔁 Scansione file .pt uno alla volta
for fname in tqdm(sorted(os.listdir(pt_dir))):
    if not fname.endswith(".pt"):
        continue

    path = os.path.join(pt_dir, fname)
    try:
        data = torch.load(path, map_location='cpu')

        # ✅ Volume e serie
        volume = data.get("volume", None)
        nomi_serie = data.get("nomi_serie", [])
        roi_masks = data.get("roi_masks", {})

        if volume is None or not isinstance(volume, torch.Tensor):
            volume_shape = "Errore"
            serie_check = {s: "Errore" for s in serie_attese}
        else:
            volume_np = volume.numpy()
            volume_shape = volume_np.shape  # (C, Z, Y, X)
            serie_check = {}
            for i, serie in enumerate(serie_attese):
                if serie in nomi_serie and i < volume_np.shape[0]:
                    val = volume_np[i]
                    if not np.all(val == -1000):
                        serie_check[serie] = "✅"
                    else:
                        serie_check[serie] = "⚠️ solo -1000"
                else:
                    serie_check[serie] = ""

        # ✅ ROI
        roi_info = []
        roi_flag = {"ROI_FLAIR": "", "ROI_MDC": ""}
        for nome, mask in roi_masks.items():
            vox = int(torch.sum(mask).item())
            roi_info.append(f"{nome} ({vox} vox)")
            if nome.upper() == "FLAIR" and vox > 0:
                roi_flag["ROI_FLAIR"] = "✅"
            if nome.upper() == "MDC" and vox > 0:
                roi_flag["ROI_MDC"] = "✅"

        records.append({
            "Paziente": fname.replace(".pt", ""),
            "Volume shape": volume_shape,
            **serie_check,
            **roi_flag
        })

    except Exception as e:
        records.append({
            "Paziente": fname.replace(".pt", ""),
            "Volume shape": "Errore",
            **{s: "Errore" for s in serie_attese},
            "ROI_FLAIR": "Errore",
            "ROI_MDC": "Errore"
        })

# ✅ Mostra la tabella finale
df = pd.DataFrame(records)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
display(df)


100%|██████████| 7/7 [02:59<00:00, 25.58s/it]


Paziente        Volume shape T1W FLAIR DWI KTRAN CBV APT ROI_FLAIR ROI_MDC
0      ID1  (6, 340, 480, 480)   ✅     ✅   ✅     ✅   ✅   ✅                  
1      ID2  (6, 340, 480, 480)   ✅     ✅   ✅     ✅   ✅   ✅                  
2      ID3  (6, 340, 560, 560)   ✅     ✅   ✅     ✅   ✅   ✅                  
3      ID5  (6, 340, 560, 560)   ✅     ✅   ✅     ✅   ✅   ✅                  
4      ID6  (6, 340, 480, 480)   ✅     ✅   ✅     ✅   ✅   ✅                  
5      ID7  (6, 340, 480, 480)   ✅     ✅   ✅     ✅   ✅   ✅